For this project you will use `requests` package and the SODA API to access data through https://data.nashville.gov/. We'll start out with something familiar, the [Top 500 Monthly Searches](https://data.nashville.gov/Public-Services/Nashville-gov-Top-500-Monthly-Searches/fuaa-r5cm), then pull in different datasets further on. You will make different API requests for each individual question.

The documentation for the SODA API is [here](https://dev.socrata.com/consumers/getting-started.html). 

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [ ]:
endpoint = 'https://data.nashville.gov/resource/fuaa-r5cm.json'

#### Question 1
Make an API request that returns the months where "fire" was searched in 2016. Which month had the most searches? 

In [ ]:
params1 = {
    'year' : 2016,
    'query_text' : 'fire'
}

In [ ]:
response1 = requests.get(endpoint, params = params1)
response1

In [ ]:
res1 = response1.json()
res1

In [ ]:
res1_df = pd.DataFrame(res1)
res1_df['query_count'] = res1_df['query_count'].astype(int)
res1_df

In [ ]:
max_month = res1_df.nlargest(1, 'query_count')
max_month

Fire was searched 47 times in September, 2016 (the most times for that year)

#### Question 2 
Make an API request that returns all the times a query was run more than 100 times in a month. How many times did this occur? 

In [ ]:
params2 = {
    '$limit' : 100000,
    '$where' : 'query_count > 100'
}

response2 = requests.get(endpoint , params=params2)
response2

In [ ]:
len(response2.json())

There are 1262 instances when a query term was search more than 100 times in a given month

#### Question 3
Make another API request that returns all the times "codes" was searched more than 100 times in a month. How many times did this occur? 

In [ ]:
params3 = {
    '$limit' : 100000,
    'query_text' : 'codes',
    '$where' : 'query_count > 100'
}

response3 = requests.get(endpoint, params=params3)
response3

In [ ]:
len(response3.json())

There are 56 instances when the term 'codes' was searched more than 100 times in a given month

#### Question 4
Make an API request that returns the entire Top 500 Monthly Searches dataset. Make a chart that shows the number of times "maps" was searched in a month across the entire time frame.

In [ ]:
params4 = {
    '$limit' : 100000
}
response4 = requests.get(endpoint, params=params4)
response4

In [ ]:
full_res = response4.json()
full_res

In [ ]:
maps_dict = {}

for line in full_res:
    text = line.get('query_text')
    mon_yr = str(line.get('month_name')+', '+line.get('year'))
    count = line.get('query_count')
    if text == 'maps':
       maps_dict[mon_yr] = count
    
maps_dict

In [ ]:
maps_df = pd.DataFrame.from_dict(maps_dict, orient='index')
maps_df = maps_df.reset_index()
maps_df.columns = ['date', 'count']
maps_df['count'] = maps_df['count'].astype(float)
maps_df['date'] = pd.to_datetime(maps_df['date'])
maps_df['mon_yr'] = maps_df['date'].dt.strftime('%m/%Y')
maps_df = maps_df.sort_values('date')
maps_df

In [ ]:
plt.figure(figsize=(16, 8))
plt.plot('date', 'count', data = maps_df)
plt.xticks(rotation = 30);

### Stretch Questions

#### Question 5
Make an API request to pull back all the data from [hubNashville (311) Service Requests](https://data.nashville.gov/Public-Services/hubNashville-311-Service-Requests/7qhx-rexh) (check to see how many rows you can return in a single request). Compare it to the Top 500 Monthly Searches data set. What do you observe? (This is open-ended, there isn't a specific answer for this one)

In [ ]:
#pulling in all 311 Service Requests, confirming my limit is above the number of requests brought in
hub_endpoint = 'https://data.nashville.gov/resource/7qhx-rexh.json'
hub_params = {
    '$limit' : 1100000
}

hub_response = requests.get(hub_endpoint, params=hub_params)

len(hub_response.json())

In [ ]:
hub_df = pd.DataFrame(hub_response.json())
hub_df[['date_time_opened', 'date_time_closed']] = hub_df[['date_time_opened', 'date_time_closed']].apply(pd.to_datetime)
hub_df.info()

In [ ]:
#simplifying the dataframe to just key columns
hub_df['open_mon_yr'] = hub_df['date_time_opened'].dt.to_period('M')
hub_df['close_mon_yr'] = hub_df['date_time_closed'].dt.to_period('M')
simp_hub_df = hub_df[['case_number', 'status', 'case_request', 'case_subrequest', 'additional_subrequest', 
                      'open_mon_yr', 'close_mon_yr']]
simp_hub_df = simp_hub_df.sort_values('open_mon_yr')
simp_hub_df

In [ ]:
#Turning the search data into a dataframe
search_df = pd.DataFrame(full_res)
search_df['mon_yr'] = search_df['month_name']+' '+search_df['year']
search_df['mon_yr'] = pd.to_datetime(search_df['mon_yr'])
search_df['mon_yr'] = search_df['mon_yr'].dt.to_period('M')
search_df.info()

In [ ]:
#Simplifying the search dataframe, cleaning up the query_text column a bit
simp_search_df = search_df[['query_text', 'mon_yr', 'query_count']]
simp_search_df['query_text'] = search_df['query_text'].str.strip(' " \'()-?').str.lower()
simp_search_df

In [ ]:
#Combining searches that were only different because of capitalization or punctuation
searches = simp_search_df.groupby(['query_text', 'mon_yr'])['query_count'].sum()
searches = pd.DataFrame(searches).reset_index()
searches['query_count'] = searches['query_count'].astype(float)
searches

In [ ]:
#Getting a count of each request type by month/year
hub_requests = simp_hub_df.groupby(['case_request', 'open_mon_yr'])['case_number'].count()
hub_requests = pd.DataFrame(hub_requests).reset_index()
hub_requests = hub_requests.rename(columns = { 'case_request' : 'request_type',
                                              'status' : 'request_count'})
hub_requests

In [ ]:
#looking specifically as searches in the first three months of the coronavirus pandemic
early_covid_searches = searches.loc[(searches['mon_yr'] >= '03/2020') &(searches['mon_yr'] <= '05-2020')]
early_covid_searches = early_covid_searches.sort_values('query_count', ascending=False)
early_covid_searches.head(50)

In [ ]:
#Just out of curiosity, looking at searches for the same 3 months the year before
pre_searches = searches.loc[(searches['mon_yr'] >= '03/2019') &(searches['mon_yr'] <= '05-2019')]
pre_searches = pre_searches.sort_values('query_count', ascending=False)
pre_searches.head(50)

In [ ]:
#Specifically looking at 311 requests from the first three months of the pandemic
early_covid_requests = hub_requests.loc[(hub_requests['open_mon_yr'] >= '03/2020') & 
                                  (hub_requests['open_mon_yr'] <= '05-2020')]
early_covid_requests = early_covid_requests.sort_values('request_count', ascending=False)
early_covid_requests.head(50)

In [ ]:
#Now pulling all COVID-19 case requests from the 311 data
all_covid_req = simp_hub_df.loc[simp_hub_df['case_request'] == 'COVID-19']
all_covid_req

In [ ]:
#Getting monthly counts of COVID-19 311 requests
monthly_covid_req = all_covid_req.groupby(['open_mon_yr'])['case_number'].count()
monthly_covid_req = pd.DataFrame(monthly_covid_req).reset_index()
monthly_covid_req.columns = ['open_yr_month', 'case_count']
monthly_covid_req['yr_mon'] = monthly_covid_req['open_yr_month'].astype(str)
monthly_covid_req

In [ ]:
#finding all COVID related searches
covid_searches = searches.loc[(searches['query_text'].str.contains('covid')) | 
                              (searches['query_text'].str.contains('corona'))]
covid_searches.info()

In [ ]:
#getting a count of all COVID reelated searches
monthly_covid_searches = covid_searches.groupby(['mon_yr'])['query_count'].sum()
monthly_covid_searches = pd.DataFrame(monthly_covid_searches).reset_index()
monthly_covid_searches['yr_mon'] = monthly_covid_searches['mon_yr'].astype(str)
monthly_covid_searches

In [ ]:
plt.figure(figsize = (18,9))
plt.plot('yr_mon', 'case_count', data = monthly_covid_req)
plt.xlabel('Month')
plt.ylabel('COVID-19 Hub Requests')
plt.xticks(rotation = 60);

In [ ]:
plt.figure(figsize = (18,9))
plt.plot('yr_mon', 'query_count', data = monthly_covid_searches)
plt.xlabel('Month')
plt.ylabel('Number of COVID-19 Related Searches')
#plt.ylim(0, 3600)
plt.xticks(rotation = 60);

#### Question 6
Find 2 new data sets on data.nashville.gov, make API requests to pull the data, and do an analysis that combines the data sets. 

In [ ]:
#Pulling in short term rental (str) permit information
str_endpoint = 'https://data.nashville.gov/resource/2z82-v8pm.json'
str_params = {
    '$limit' : 15000
}

str_response = requests.get(str_endpoint, params= str_params)
str_response

In [ ]:
str_permits = pd.DataFrame(str_response.json())
str_permits

In [ ]:
#pulling in property violation information
prop_endpoint = 'https://data.nashville.gov/resource/479w-kw2x.json'
prop_params = {
    '$limit' : 110000
}

prop_response = requests.get(prop_endpoint, params=prop_params)
prop_response

In [ ]:
prop_violations = pd.DataFrame(prop_response.json())
prop_violations

In [ ]:
#Tidying up the dataframe
prop_violations = prop_violations.drop(columns = ['council_district', ':@computed_region_wvby_4s8j', 
                                                  ':@computed_region_3aw5_2wv7', 
                                                  ':@computed_region_p6sk_2acq', 
                                                  ':@computed_region_gxvr_9jxz', 
                                                  ':@computed_region_gisn_y5cm', 
                                                  ':@computed_region_sjpq_96s8', 
                                                  ':@computed_region_v3ji_vzam', 
                                                  ':@computed_region_c9xn_skx3', 
                                                  ':@computed_region_f73m_vb2k', 
                                                  ':@computed_region_kh5x_g7w5', 
                                                  ':@computed_region_yf9r_ed6g', 
                                                  ':@computed_region_fvtq_wnma', 
                                                  ':@computed_region_b9k3_hpc2', 
                                                  ':@computed_region_cfa7_hbpz' ])
prop_violations = prop_violations.rename(columns = {'property_address' : 'address'})

prop_violations.info()

In [ ]:
#Tidying up the dataframe
str_permits = str_permits.drop(columns = ['parcel', 'census_tract', 
                                          ':@computed_region_p6sk_2acq', 
                                          ':@computed_region_gxvr_9jxz', 
                                          ':@computed_region_wvby_4s8j', 
                                          ':@computed_region_3aw5_2wv7', 
                                          ':@computed_region_cfa7_hbpz', 
                                          ':@computed_region_sjpq_96s8', 
                                          ':@computed_region_f73m_vb2k', 
                                          ':@computed_region_c9xn_skx3', 
                                          ':@computed_region_gisn_y5cm', 
                                          ':@computed_region_v3ji_vzam'])
str_permits.info()

In [ ]:
#Merging the dataframes, keeping all str properties, but only prop violations that occured at strs
str_pv = str_permits.merge(prop_violations, how='left', on=['address', 'city', 'state', 'zip'])
str_pv

In [ ]:
#Further trimming down columns
str_pv = str_pv[['applicant', 'permit', 'permit_subtype_description', 'permit_type', 'permit_subtype',
                'permit_status', 'date_entered', 'date_issued', 'expiration_date', 'address', 'city', 
                'state', 'zip', 'purpose', 'permit_owner_name', 'contact', 'council_dist', 
                'mapped_location_x', 'mapped_location_y', 'request', 'date_received', 
                'property_owner', 'complaint_source', 'reported_problem', 'violations_noted', 
                'status', 'last_activity_date', 'last_activity', 'last_activity_result']]
str_pv

In [ ]:
#dropping down just to those str properties with violations
str_violators = str_pv.loc[~str_pv['request'].isna()]
str_violators

In [ ]:
#adjusting some columns, cleaning up some of the entries
str_violators[['date_issued', 'date_received']] = str_violators[['date_issued', 'date_received']].apply(pd.to_datetime)
str_violators.loc[:, 'violations_noted'] = str_violators['violations_noted'].str.upper()
str_violators = str_violators.replace({'EXCESSIVE NOISE/WASTE' : 'EXCESSIVE NOISE / WASTE'})

In [ ]:
#Now finding those instances when the violation occurred AFTER the str permit was issued
str_pv1 = str_violations.loc[str_violations['date_issued'] <= str_violations['date_received']]
str_pv1

In [ ]:
#How many unique str properties had violations after getting the str permit 2957
num_prop_violators = len(set(str_pv1['permit']))
num_prop_violators

In [ ]:
#Looking at instances when the property had violations prior to getting the permit
str_pv2 = str_violations.loc[str_violations['date_issued'] > str_violations['date_received']]
str_pv2

In [ ]:
#How many unique str properties had violations prior to the permit being issued? 549
num_pre_violators = len(set(str_pv2['permit']))
num_pre_violators

In [ ]:
#Now looking at repeat offenders for those str properties with violations after getting their permits
repeats = (
    str_pv1.groupby(['permit'])['request'].count().to_frame().reset_index()
repeats.columns = ['str_permit', 'property_complaints']
repeats = repeats.loc[repeats['property_complaints'] > 1].sort_values('property_complaints', ascending=False)

repeats.head(50)

In [ ]:
#Checking out post-permit violations by str permit subtype
pv_by_subtype = str_pv1.groupby(['permit_subtype_description'])['request'].count()
pv_by_subtype = pd.DataFrame(pv_by_subtype).reset_index()
pv_by_subtype.columns = ['str_permit_subtype', 'property_complaints']
pv_by_subtype

In [ ]:
#Looking at those str permit applications that resulted in a permit being issued
issued_str = str_permits.loc[~str_permits['date_issued'].isna()].reset_index()
issued_str.info()

In [ ]:
#str permit subtypes for those properties that received permits
str_by_subtype = issued_str.groupby(['permit_subtype_description'])['permit'].count()
str_by_subtype = pd.DataFrame(str_by_subtype).reset_index()
str_by_subtype.columns = ['str_permit_subtype', 'total_permits']
str_by_subtype

In [ ]:
sns.set_theme(style='whitegrid')
pb = sns.barplot(x="total_permits", y="str_permit_subtype", data = str_by_subtype, palette='Set2')
plt.title('Short Term Rental Properties by Permit Subtype')
plt.xlabel('Total Permits')
plt.ylabel('Permit Subtype')
pb.figure.set_size_inches(6, 4);

In [ ]:
sns.set_theme(style='whitegrid')
pvb = sns.barplot(x='property_complaints', y='str_permit_subtype', data = pv_by_subtype, palette='Set2')
plt.title('Short Term Rental Properties with Property Standards Complaints by Permit Subtype')
plt.xlabel('Total Complaints')
plt.ylabel('Permit Subtype')
pvb.set(xlim=(0, 6000))
pvb.figure.set_size_inches(6, 4);

In [ ]:
#top 10 str post-permit violations by types of violation
str_pv_by_violations = str_pv1.groupby(['violations_noted'])['request'].count()
str_pv_by_violations = pd.DataFrame(str_pv_by_violations).reset_index()
str_pv_by_violations.columns = ['violation', 'property_complaints']
str_pv_by_violations = str_pv_by_violations.sort_values('property_complaints', ascending=False)
top_str_pv_by_violations = str_pv_by_violations.head(10)
top_str_pv_by_violations

In [ ]:
#top 10 overall property violations by types of violation
prop_violations['violations_noted'] = prop_violations['violations_noted'].str.upper()
prop_violations = prop_violations.replace({'EXCESSIVE NOISE/WASTE' : 'EXCESSIVE NOISE / WASTE'})
pv_by_violations = prop_violations.groupby(['violations_noted'])['request'].count()
pv_by_violations = pd.DataFrame(pv_by_violations).reset_index()
pv_by_violations.columns = ['violation', 'property_complaints']
pv_by_violations = pv_by_violations.sort_values('property_complaints', ascending=False)
pv_by_violations.head(10)

In [ ]:
sns.set_theme(style='whitegrid')
svb = sns.barplot(x='property_complaints', y='violation', data = pv_by_violations.head(10), palette='cubehelix')
plt.title('All Property Standards Complaints by Violation Type - Top 10')
plt.xlabel('Total Complaints')
plt.ylabel('Violation')
svb.set(xlim=(0, 6500))
svb.figure.set_size_inches(12, 8);

In [ ]:
sns.set_theme(style='whitegrid')
avb = sns.barplot(x='property_complaints', y='violation', data = top_str_pv_by_violations, palette='icefire_r')
plt.title('Short Term Rental Properties with Property Standards Complaints by Violation Type - Top 10')
plt.xlabel('Total Complaints')
plt.ylabel('Violation')
avb.set(xlim=(0, 6500))
avb.figure.set_size_inches(12, 8);

In [ ]:
#post-permit str property violations by zip code
pv_by_zip = str_pv1.groupby(['zip'])['request'].count()
pv_by_zip = pd.DataFrame(pv_by_zip).reset_index()
pv_by_zip.columns = ['str_zipcode', 'property_complaints']
pv_by_zip = pv_by_zip.sort_values('property_complaints', ascending=False)
pv_by_zip

In [ ]:
#issued str permits by zipcode
str_by_zip = issued_str.groupby(['zip'])['permit'].count()
str_by_zip = pd.DataFrame(str_by_zip).reset_index()
str_by_zip.columns = ['str_zipcode', 'total_permits']
str_by_zip = str_by_zip.sort_values('total_permits', ascending=False)
str_by_zip

In [ ]:
#creating a df with str permit and str violation info, calculating proportions of each
zip_pv_proportions = pd.merge(str_by_zip, pv_by_zip)
zip_pv_proportions['proportion_complaints'] = round((zip_pv_proportions['property_complaints']/
                                                     zip_pv_proportions['property_complaints'].sum())*100, 1)
zip_pv_proportions['proportion_permits'] = round((zip_pv_proportions['total_permits']/
                                                     zip_pv_proportions['total_permits'].sum())*100, 1)
zip_pv_proportions = zip_pv_proportions.sort_values('proportion_complaints', ascending=False)
zip_pv_proportions

In [ ]:
#scatter plot comparing number of str permits and number of str property violations
(
    sns.relplot(data=zip_pv_proportions.head(10), 
                x="total_permits", 
                y="property_complaints", 
                hue="str_zipcode", 
                palette="cubehelix")
);

In [ ]:
#comparing proportion of total str permits to proportion of total str property violations
(
    zip_pv_proportions.head(10).plot(x="str_zipcode", 
                                     y=["proportion_permits", 
                                        "proportion_complaints"], 
                                     kind="barh",
                                     figsize=(9,8))
);